In [1]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
from keras.utils.np_utils import to_categorical
import matplotlib.pyplot as plt
import math
import cv2

Using TensorFlow backend.


In [2]:
img_width, img_height = 150,150

top_model_weights_path = '/data/Documents/model.h5'
train_data_dir = '/data/Documents/6_classes_Naveen/training'
validation_data_dir = '/data/Documents/6_classes_Naveen/testing'

In [4]:
# number of epochs to train top model
epochs = 10
# batch size used by flow_from_directory and predict_generator
batch_size = 16


In [7]:

def save_features():
    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    datagen = ImageDataGenerator(rescale=1. / 255)

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

    print(len(generator.filenames))
    print(generator.class_indices)
    print(len(generator.class_indices))

    nb_train_samples = len(generator.filenames)
    num_classes = len(generator.class_indices)

    predict_size_train = int(math.ceil(nb_train_samples / batch_size))

    bottleneck_features_train = model.predict_generator(
        generator, predict_size_train)

    np.save('6Class_features_train.npy', bottleneck_features_train)

    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

    nb_validation_samples = len(generator.filenames)

    predict_size_validation = int(
        math.ceil(nb_validation_samples / batch_size))

    bottleneck_features_validation = model.predict_generator(
        generator, predict_size_validation)

    np.save('6Class_features_validation.npy',
            bottleneck_features_validation)

In [12]:
def train_top_model():
    datagen_top = ImageDataGenerator(rescale=1. / 255)
    generator_top = datagen_top.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

    nb_train_samples = len(generator_top.filenames)
    num_classes = len(generator_top.class_indices)

    # save the class indices to use use later in predictions
    np.save('6class_indices.npy', generator_top.class_indices)

    # load the bottleneck features saved earlier
    train_data = np.load('6Class_features_train.npy',allow_pickle=True)

    # get the class lebels for the training data, in the original order
    train_labels = generator_top.classes

    train_labels = to_categorical(train_labels, num_classes=num_classes)

    generator_top = datagen_top.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

    nb_validation_samples = len(generator_top.filenames)

    validation_data = np.load('6Class_features_validation.npy',allow_pickle=True)

    validation_labels = generator_top.classes
    validation_labels = to_categorical(
        validation_labels, num_classes=num_classes)

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels))

   
    model.save_weights(top_model_weights_path)

    (eval_loss, eval_accuracy) = model.evaluate(
        validation_data, validation_labels, batch_size=batch_size, verbose=1)

    print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))
    print("[INFO] Loss: {}".format(eval_loss))

In [13]:
save_features()
train_top_model()

Found 2572 images belonging to 6 classes.
2572
{'Looking_at_mirrior': 0, 'drinking': 1, 'focused': 2, 'music': 3, 'reaching_behind': 4, 'talking_in phone': 5}
6
Found 632 images belonging to 6 classes.
Found 2572 images belonging to 6 classes.
Found 632 images belonging to 6 classes.
Train on 2572 samples, validate on 632 samples
Epoch 1/10
2572/2572 [==============================] - 7s 3ms/step - loss: 1.6735 - accuracy: 0.2920 - val_loss: 0.9953 - val_accuracy: 0.7437
Epoch 2/10
2572/2572 [==============================] - 7s 3ms/step - loss: 0.5724 - accuracy: 0.8095 - val_loss: 0.4345 - val_accuracy: 0.8608
Epoch 3/10
2572/2572 [==============================] - 7s 3ms/step - loss: 0.2140 - accuracy: 0.9265 - val_loss: 0.7020 - val_accuracy: 0.8291
Epoch 4/10
2572/2572 [==============================] - 7s 3ms/step - loss: 0.1219 - accuracy: 0.9650 - val_loss: 0.4816 - val_accuracy: 0.8797
Epoch 5/10
2572/2572 [==============================] - 7s 3ms/step - loss: 0.0830 - accurac

In [14]:
#for images
def predict():
    # load the class_indices saved in the earlier step
    class_dictionary = np.load('6class_indices.npy',allow_pickle=True).item()

    num_classes = len(class_dictionary)

    # add the path to your test image below
    image_path ='/data/Documents/Driver distraction data/RGB1/S23/AC7/RGB1_SUB23ACT7F4.jpg'

    orig = cv2.imread(image_path)

    print("[INFO] loading and preprocessing image...")
    image = load_img(image_path, target_size=(150,150))
    image = img_to_array(image)

    # important! otherwise the predictions will be '0'
    image = image / 255

    image = np.expand_dims(image, axis=0)

    
    model = applications.VGG16(include_top=False, weights='imagenet')

    
    bottleneck_prediction = model.predict(image)

   
    model = Sequential()
    model.add(Flatten(input_shape=bottleneck_prediction.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='sigmoid'))

    model.load_weights(top_model_weights_path)

    class_predicted = model.predict_classes(bottleneck_prediction)

    

    inID = class_predicted[0]

    inv_map = {v: k for k, v in class_dictionary.items()}

    label = inv_map[inID]

  
    print("Image ID: {}, Label: {}".format(inID, label))

    
    cv2.putText(orig, "Predicted: {}".format(label), (10, 30),
                cv2.FONT_HERSHEY_PLAIN, 1.5, (43, 99, 255), 2)

    cv2.imshow("Classification", orig)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


predict()

cv2.destroyAllWindows()

[INFO] loading and preprocessing image...
Image ID: 5, Label: talking_in phone


In [1]:
#for offline video
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras import applications
from keras.layers import Dropout, Flatten, Dense
import cv2
import re
from os.path import isfile, join
top_model_weights_path = 'model.h5'

def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ] 
    return sorted(data, key=alphanum_key)

vidcap = cv2.VideoCapture('Vedio3.mp4')
targetdir='/data/video_now/'
os.mkdir(targetdir)

def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = vidcap.read()
    if hasFrames:
        cv2.imwrite(os.path.join(targetdir,"input_image"+str(count_input)+".jpg"), image)     # save frame as JPG file
    return hasFrames
sec = 0
frameRate = 1 #//it will capture image in each 0.5 second
count_input=1
success = getFrame(sec)
while success:
    count_input = count_input + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)



#count=0
def predict():
    # load the class_indices saved in the earlier step
    class_dictionary = np.load('6class_indices.npy',allow_pickle=True).item()

    num_classes = len(class_dictionary)
    folder = '/data/video_now/'
    #image_path='/data/EMBITEL/emb-bhuvpra/Documents/data1/test/focused/RGB1_SUB1ACT1F92.jpg'
    count=0
    # add the path to your test image below
    for filename in sorted_alphanumeric(os.listdir(folder)):
        
        image_path=os.path.join(folder,filename)
        orig = cv2.imread(image_path)
    

    #orig = cv2.imread(image_path)

        print("[INFO] loading and preprocessing image...")
        image = load_img(image_path, target_size=(150,150))
        image = img_to_array(image)        
        # important! otherwise the predictions will be '0'
        image = image / 255



        image = np.expand_dims(image, axis=0)


        model = applications.VGG16(include_top=False, weights='imagenet')


        bottleneck_prediction = model.predict(image)


        model = Sequential()
        model.add(Flatten(input_shape=bottleneck_prediction.shape[1:]))
        model.add(Dense(256, activation='relu'))
        model.add(Dropout(0.5))
        model.add(Dense(num_classes, activation='sigmoid'))

        model.load_weights(top_model_weights_path)

        class_predicted = model.predict_classes(bottleneck_prediction)



        inID = class_predicted[0]

        inv_map = {v: k for k, v in class_dictionary.items()}

        label = inv_map[inID]


        print("Image ID: {}, Label: {}".format(inID, label))


        cv2.putText(orig, "Predicted: {}".format(label), (15, 15),
                    cv2.FONT_HERSHEY_PLAIN, 1, (43, 99, 255), 2)

        #cv2.imshow("Classification", orig)
        cv2.imwrite('/data/outputfile2/output_image{}.jpg'.format(count),orig)
         
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
        count+=1
    


predict()

pathIn= '/data/outputfile2/'
pathOut = 'output_actions2now.avi'
fps = 1
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
files.sort()
frame_array = []
files = [f for f in os.listdir(pathIn) if isfile(join(pathIn, f))]
#for sorting the file names properly
files.sort(key = lambda x: x[5:-4])
for i in range(len(files)):
    filenamepred=pathIn + files[i]
    #reading each files
    img = cv2.imread(filenamepred)
    height, width, layers = img.shape
    size = (width,height)
    
    #inserting the frames into an image array
    frame_array.append(img)
out = cv2.VideoWriter(pathOut,cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
for i in range(len(frame_array)):
    # writing to a image array
    out.write(frame_array[i])
out.release()

cv2.destroyAllWindows()

Using TensorFlow backend.


[INFO] loading and preprocessing image...
Image ID: 0, Label: Looking_at_mirrior
[INFO] loading and preprocessing image...
Image ID: 0, Label: Looking_at_mirrior
[INFO] loading and preprocessing image...
Image ID: 2, Label: focused
[INFO] loading and preprocessing image...
Image ID: 3, Label: music
[INFO] loading and preprocessing image...
Image ID: 3, Label: music
[INFO] loading and preprocessing image...
Image ID: 3, Label: music
[INFO] loading and preprocessing image...
Image ID: 5, Label: talking_in phone
[INFO] loading and preprocessing image...
Image ID: 3, Label: music
[INFO] loading and preprocessing image...
Image ID: 3, Label: music
[INFO] loading and preprocessing image...
Image ID: 5, Label: talking_in phone
